In [2]:
# Instala a biblioteca oficial da Groq, que permite acessar os modelos LLM
# disponíveis na nuvem da Groq (como LLaMA 3) de forma rápida e otimizada.
!pip install -q groq

# Instala a biblioteca "wikipedia-api", que facilita buscar e extrair textos
# diretamente da Wikipedia em diferentes idiomas.
!pip install -q wikipedia-api

# Instala a biblioteca "sentence-transformers", que fornece modelos prontos
# para gerar embeddings de frases e textos. Esses embeddings permitem realizar
# busca semântica, similaridade textual e são fundamentais em arquiteturas RAG.
!pip install -q sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.3 MB/s eta 0:00:00


In [3]:
# Biblioteca NumPy, usada para manipulação de arrays, cálculos numéricos
# e operações matemáticas que podem ser úteis no processamento dos embeddings.
import numpy as np

# Biblioteca requests, usada para fazer requisições HTTP.
# Pode ser útil caso você precise buscar dados de APIs externas além da Wikipedia.
import requests

# Biblioteca textwrap, utilizada para formatar e quebrar textos em blocos menores.
# Isso ajuda na hora de exibir trechos recuperados da Wikipedia de forma legível.
import textwrap

# Módulo do Google Colab que permite acessar variáveis de ambiente salvas em userdata.
# Aqui é usado geralmente para armazenar a API Key da Groq de forma segura.
from google.colab import userdata

# Cliente oficial da Groq, usado para interagir com os modelos LLM hospedados na Groq Cloud.
from groq import Groq

# Biblioteca SentenceTransformer, usada para gerar embeddings semânticos de sentenças e textos.
# Essencial para a etapa de recuperação (retrieval) na arquitetura RAG.
from sentence_transformers import SentenceTransformer

# Biblioteca Wikipedia API, que permite buscar e extrair conteúdo da Wikipedia
# em diferentes idiomas, fornecendo a base de conhecimento para o RAG.
from wikipediaapi import Wikipedia

In [5]:
# Inicializa o cliente da Groq usando a chave de API armazenada no Google Colab.
# O método userdata.get("GROQ_API_KEY") busca a variável segura que você salvou
# no Colab (Settings > Secrets), evitando expor a chave diretamente no código.
client = Groq(api_key=userdata.get("GROQ_API_KEY"))

In [34]:
# Inicializa a instância da Wikipedia
# "RAGBot/0.0" é o user-agent que identifica o seu bot para a Wikipedia
# "pt" indica que vamos usar a versão em português da Wikipedia
wiki = Wikipedia("RAGBot/0.0", "pt")

# Busca a página "Ayrton Senna" e pega todo o texto dela
doc = wiki.page("Faustão").text

# Divide o texto em parágrafos, usando duplas quebras de linha como separador
# Isso ajuda a criar "chunks" menores de texto para a etapa de recuperação (retrieval)
paragraphs = doc.split("\n\n")

In [35]:
# Loop para percorrer todos os parágrafos recuperados da Wikipedia
for i, p in enumerate(paragraphs):

  # Formata o texto para quebrar linhas a cada 100 caracteres
  # Isso deixa a exibição mais organizada no notebook
  wrapped_text = textwrap.fill(p, width=100)

  # Imprime uma linha separadora antes do parágrafo
  print("-" * 50)

  # Imprime o parágrafo formatado
  print(wrapped_text)

  # Imprime outra linha separadora depois do parágrafo
  print("-" * 50)

--------------------------------------------------
Fausto Corrêa da Silva (São Paulo, 2 de maio de 1950), popularmente conhecido como Faustão, é um
apresentador, jornalista e radialista brasileiro. É considerado um dos maiores animadores da
história da televisão no Brasil. Iniciou a carreira na rádio como jornalista esportivo, tendo
posteriormente seguido para a televisão, onde consagrou-se como apresentador e conquistou uma legião
de fãs devido ao seu carisma, sinceridade e bom humor. Nascido na cidade de São Paulo em 1950, morou
durante a infância em várias cidades do interior paulista, até se mudar com a família para Campinas,
onde teve suas primeiras experiências como locutor nas rádios em 1964. Durante a década de 1960,
trabalhou assiduamente como repórter esportivo nas rádios Excelsior e Globo, cobrindo os jogos
narrados por Osmar Santos. No ramo esportivo, chegou a viajar para a América do Sul, Europa e Arábia
Saudita, cobrindo duas Copas do Mundo e várias edições da Copa Libert

In [36]:
# Inicializa o modelo de embeddings da SentenceTransformer
# "Alibaba-NLP/gte-base-en-v1.5" é o modelo usado para gerar vetores semânticos de alta qualidade
# trust_remote_code=True permite carregar código remoto do modelo caso necessário
embedding_model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

# Gera embeddings para todos os parágrafos da Wikipedia
# normalize_embeddings=True faz a normalização dos vetores, útil para cálculo de similaridade por cosseno
embedded_paragraphs = embedding_model.encode(paragraphs, normalize_embeddings=True)

# Mostra a quantidade de parágrafos e o tamanho dos vetores gerados
# embedded_paragraphs.shape retorna (num_paragraphs, embedding_dimension)
print("Number of paragraphs and vector sizes: ", embedded_paragraphs.shape)

Number of paragraphs and vector sizes:  (22, 768)


In [37]:
# Define a pergunta que queremos responder usando o conteúdo da Wikipedia
query = "Quando o nosso querido Faustão foi trabalhar na Globo?"

# Gera o embedding (vetor semântico) da pergunta usando o mesmo modelo que usamos para os parágrafos
# normalize_embeddings=True garante que o vetor esteja normalizado para cálculo de similaridade por cosseno
query_embedded = embedding_model.encode(query, normalize_embeddings=True)

# Mostra o formato do vetor gerado
# Normalmente será (embedding_dimension,), compatível com os vetores dos parágrafos
query_embedded.shape

(768,)

In [38]:
# Calcula a similaridade entre a pergunta e cada parágrafo usando produto escalar
# Aqui estamos usando np.dot como alternativa à similaridade de cosseno
# embedded_paragraphs: shape (num_paragraphs, embedding_dim)
# query_embedded.T: shape (embedding_dim,)
# O resultado será um vetor com uma pontuação para cada parágrafo
similarities = np.dot(embedded_paragraphs, query_embedded.T)

# Mostra o formato do vetor de similaridades
# Espera-se (num_paragraphs,), ou seja, uma pontuação para cada parágrafo
print(similarities.shape)

# Exibe as pontuações de similaridade de todos os parágrafos
similarities

(22,)


array([0.6450616 , 0.4713364 , 0.49568436, 0.5731863 , 0.50807613,
       0.660562  , 0.61118793, 0.57909775, 0.54875356, 0.5031643 ,
       0.523284  , 0.555702  , 0.57654464, 0.51517695, 0.58812165,
       0.6359531 , 0.48171517, 0.5156719 , 0.3162569 , 0.47418648,
       0.36744577, 0.444391  ], dtype=float32)

In [39]:
# Encontra os índices dos 3 parágrafos mais relevantes
# np.argsort retorna os índices que ordenariam o array de similaridades
# [-3:] pega os 3 últimos (maior similaridade) e [::-1] inverte para ordem decrescente
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

# Recupera os parágrafos correspondentes aos 3 índices mais relevantes
most_similiar_paragraphs = [paragraphs[idx] for idx in top_3_idx]

In [40]:
# Inicializa uma variável para armazenar todo o contexto que será passado para o modelo
context = ""

# Percorre os parágrafos mais relevantes (top 3)
for i, p in enumerate(most_similiar_paragraphs):

  # Formata o parágrafo para quebrar linhas a cada 100 caracteres
  wrapped_text = textwrap.fill(p, width=100)

  # Imprime uma linha separadora antes do parágrafo
  print("-" * 50)

  # Exibe o parágrafo numerado e formatado
  print(f"Paragraph {i + 1}: ", wrapped_text)

  # Imprime outra linha separadora depois do parágrafo
  print("-" * 50)

  # Adiciona o parágrafo ao contexto final, separando-os com duas quebras de linha
  # Esse contexto será usado pelo modelo LLM (Groq) para gerar a resposta
  context += wrapped_text + "\n\n"

--------------------------------------------------
Paragraph 1:  Domingão do Faustão Em 26 de março de 1989, estreou na TV Globo como apresentador do programa de
auditório Domingão do Faustão, nas tardes de domingo, com a participação de Caçulinha e Lucimara
Parisi até 2009. O programa foi um dos que ficou mais tempo em produção pela emissora. Em maio de
2008, chegou à milésima edição, sendo uma das atrações de maior faturamento para o canal. Em maio de
2021 Fausto Silva assinou um contrato de cinco anos com a Rede Bandeirantes, com estreia prevista
para janeiro de 2022. Em 17 de junho de 2021, a Globo anunciou a saída antecipada de Fausto Silva,
que seria substituído por Tiago Leifert até janeiro, quando Luciano Huck assumiria o comando do
programa, porém o início de Huck foi antecipado, e o apresentador estreou em 5 de setembro de 2021.
--------------------------------------------------
--------------------------------------------------
Paragraph 2:  Fausto Corrêa da Silva (São Paulo

In [41]:
# Cria o prompt que será enviado ao modelo Groq
# O prompt inclui:
# 1. Instruções claras para o modelo (responder baseado no contexto, não inventar respostas)
# 2. O contexto formado pelos parágrafos mais relevantes
# 3. A pergunta que queremos responder
prompt = f"""
      Baseando-se no contexto abaixo, responda a pergunta.
      Se você não souber a resposta, diga que não sabe. Não tente inventar uma resposta.

      Contexto: {context}

      ---

      Pergunta: {query}
"""

# Chama o modelo Groq para gerar a resposta
# model="moonshotai/kimi-k2-instruct" é o modelo LLM usado
# messages: formato de chat, aqui apenas com a mensagem do usuário
chat_completion = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct",
    messages=[
        {"role": "user",
         "content": prompt}
    ],
)

# Recupera a resposta gerada pelo modelo e remove espaços extras
response = chat_completion.choices[0].message.content.strip()

# Exibe a resposta final do modelo
print(response)

Faustão foi trabalhar na Globo em 1989.
